In [1]:
# SI-model with amyloid

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import glob
from scipy import stats

# Reading in all the tau PET data provided from the github, as well as taking the mean of the different stages.
df = pd.read_excel("Filepath/PET TAU/AD_pos.xlsx")
df = df.iloc[:, 3:]            
AD_pos = df.to_numpy()  # shape (number of subjects, number of brain regions).

df = pd.read_excel("Filepath/PET TAU/CN_pos.xlsx")
df = df.iloc[:, 3:]
CN_pos = df.to_numpy()

df = pd.read_excel("Filepath/PET TAU/CN_neg.xlsx")
df = df.iloc[:, 3:]
CN_neg = df.to_numpy()

df = pd.read_excel("Filepath/PET TAU/MCI_pos.xlsx")
df = df.iloc[:, 3:]
MCI_pos = df.to_numpy()

# This part was to check the correlation between each group stage.
CN_neg_all = CN_neg.mean(axis=0)
CN_pos_all = CN_pos.mean(axis=0)
MCI_pos_all = MCI_pos.mean(axis=0)
AD_pos_all = AD_pos.mean(axis=0)  

r_CN_AD  = np.corrcoef(CN_pos_all,  AD_pos_all)[0, 1]
r_MCI_AD = np.corrcoef(MCI_pos_all, AD_pos_all)[0, 1]
r_CN_MCI = np.corrcoef(CN_pos_all,  MCI_pos_all)[0, 1]
print(r_CN_AD, r_MCI_AD, r_CN_MCI)

# Reading in all amyloid PET data for usage of regional vulnability.
df = pd.read_excel("Filepath/PET AMYLOID/CN_pos.xlsx")
df = df.iloc[:, 3:]
amy_CN_pos = df.to_numpy()

df = pd.read_excel("Filepath/PET AMYLOID/MCI_pos.xlsx")
df = df.iloc[:, 3:]
amy_MCI_pos = df.to_numpy()

df = pd.read_excel("Filepath/PET AMYLOID/AD_pos.xlsx")
df = df.iloc[:, 3:]
amy_AD_pos = df.to_numpy()

# Mean over all stages.
amy_all = np.vstack([amy_CN_pos, amy_MCI_pos, amy_AD_pos])  
amy_mean = amy_all.mean(axis=0)                              

# This part normlises the values from 0 to 1. 
amy_norm = (amy_mean - amy_mean.min()) / (amy_mean.max() - amy_mean.min() + 1e-12)

# Creating of SC-network. 
CN_neg_folder = "Filepath/DTI/CN_neg" 

files = glob.glob(os.path.join(CN_neg_folder, "*.xlsx"))
print(f"Found {len(files)} files")
if len(files) == 0:
    raise SystemExit("No .xlsx-filer found")

sum_mat = None
shape_ref = None

for f in files:
    M = pd.read_excel(f, header=None).values

    if sum_mat is None:
        sum_mat = np.zeros_like(M, dtype=float)
        shape_ref = M.shape
    else:
        assert M.shape == shape_ref, f"Wrong shape in {f}: {M.shape} != {shape_ref}"

    sum_mat += M

SC_network = sum_mat / len(files)
print("Shape on SC_network:", SC_network.shape)

0.9507412509120934 0.9763157619890355 0.973832136100781
Found 640 files
Shape on SC_network: (86, 86)


In [13]:
# The SI-modell.
def SI_simulation_step(SC_network, status, beta, vulnerability=None):
    """
    status: arrays of 0/1 (susceptible/infected).
    beta: infection rate.
    vulnerability: regional vulnability of all 86 brain regions. array (N_regions,).
    
    """
    new_status = status.copy()
    infected = np.where(status == 1)[0]

    for i in infected:
        neighbours = np.where(SC_network[i, :] > 0)[0]
        if neighbours.size == 0:
            continue

        susceptible_neighbours = neighbours[status[neighbours] == 0]
        if susceptible_neighbours.size == 0:
            continue

        prob_infection_for_ij = beta * SC_network[i, susceptible_neighbours]

        # Vulnerability for target node j.
        if vulnerability is not None:
            prob_infection_for_ij = prob_infection_for_ij * vulnerability[susceptible_neighbours]

        # So that the probability doesn't become more than 1.
        prob_infection_for_ij = np.clip(prob_infection_for_ij, 0.0, 1.0)

        random_numbers = np.random.random(prob_infection_for_ij.size)
        newly_infected = susceptible_neighbours[random_numbers < prob_infection_for_ij]
        new_status[newly_infected] = 1

    return new_status

# Simulation.
N_regions = SC_network.shape[0]

# The different parameter values in the sweep to find the optimal parameter values for infection rate beta and vulnerability. 
beta_list = np.logspace(-6, -1, 10)
v_min_list  = [0.3, 0.5, 0.7]          
v_max_list  = [1.3, 1.5, 1.7]
                       
iterations = 500                  
n_runs = 200
search_seed = 0 
#best_correlation_per_subject = []
#best_p_per_subject = []

# Best so far. 
best_corr_AD = -np.inf
best_beta_AD = None
best_vmin_AD = None
best_vmax_AD = None
best_t_AD = None
best_pattern_AD = None   
best_infection_prob = None   
best_p = None

for v_min in v_min_list:
        for v_max in v_max_list:
            
            # Creating a vulnablity factor (vulnability for each brain regions).
            vulnerability = v_min + (v_max - v_min) * amy_norm 
            
            for beta in beta_list:
                print(f"Simulating beta={beta:.2e}, v_min={v_min}, v_max={v_max}", end="\r")
                
                # Fixating the seed for this combination of paramters.
                np.random.seed(search_seed)

                status_history = np.zeros((n_runs, iterations, N_regions))

                for n in range(n_runs):
                    status = np.zeros(N_regions, dtype=int)
                    starting_region = [40, 41, 42, 43]  # PH and hippocampus.
                    status[starting_region] = 1

                    status_history[n, 0, :] = status
                    for t in range(1, iterations):
                        status = SI_simulation_step(SC_network, status, beta, vulnerability=vulnerability)
                        status_history[n, t, :] = status

                # Mean over all runs to get a good value on infection_probability.
                infection_probability = status_history.mean(axis=0)  
                
                # for individual in range(len(AD_pos)):
                for t in range(iterations):
                    sim_vec = infection_probability[t, :]
                    if sim_vec.std() == 0 or AD_pos_all.std() == 0:
                        corr = 0.0
                    else:
                        corr, p = stats.pearsonr(sim_vec, AD_pos_all)

                    if corr > best_corr_AD:
                        best_corr_AD = corr
                        best_beta_AD = beta
                        best_vmin_AD = v_min
                        best_vmax_AD = v_max
                        best_t_AD = t
                        best_pattern_AD = sim_vec.copy()
                        best_infection_prob = infection_probability.copy()
                        best_p = p
                #best_correlation_per_subject.append(best_corr_AD)
                #best_p_per_subject.append(best_p)

print("\nBest values against AD_positive:")
print(f"  beta = {best_beta_AD:.2e}, t = {best_t_AD}, v_min = {best_vmin_AD}, v_max = {best_vmax_AD}, R = {best_corr_AD:.3f}, p = {best_p:.5f}")

Simulating beta=1.00e-01
Best values against AD_positive:
  beta = 7.74e-03, t = 4, v_min = None, v_max = None, R = 0.208, p = 0.05510


In [1]:
# The perfomance for all three positive stages of AD (CN_pos_all, MCI_pos_all and AD_pos_all).
infection_probability = best_infection_prob 

corr_CN_t   = np.zeros(iterations)
corr_MCI_t  = np.zeros(iterations)
corr_AD_t   = np.zeros(iterations)

for t in range(iterations):
    sim_vec = infection_probability[t, :]  

    # CN+.
    if sim_vec.std() == 0 or CN_pos_all.std() == 0:
        corr_CN_t[t] = 0.0
    else:
        corr_CN_t[t] = np.corrcoef(sim_vec, CN_pos_all)[0, 1]

    # MCI+.
    if sim_vec.std() == 0 or MCI_pos_all.std() == 0:
        corr_MCI_t[t] = 0.0
    else:
        corr_MCI_t[t] = np.corrcoef(sim_vec, MCI_pos_all)[0, 1]

    # AD+.
    if sim_vec.std() == 0 or AD_pos_all.std() == 0:
        corr_AD_t[t] = 0.0
    else:
        corr_AD_t[t] = np.corrcoef(sim_vec, AD_pos_all)[0, 1]

t_peak_CN   = np.argmax(corr_CN_t)
t_peak_MCI  = np.argmax(corr_MCI_t)
t_peak_AD   = np.argmax(corr_AD_t)

R_peak_CN   = corr_CN_t[t_peak_CN]
R_peak_MCI  = corr_MCI_t[t_peak_MCI]
R_peak_AD   = corr_AD_t[t_peak_AD]

NameError: name 'best_infection_prob' is not defined

In [ ]:
# Robustcheck.

seed_list = [0, 1, 2, 3, 4, 5, 6, 7, 8]   
R_peaks = []

for s in seed_list:
    np.random.seed(s)

    vulnerability_best = best_vmin_AD + (best_vmax_AD - best_vmin_AD) * amy_norm

    status_history = np.zeros((n_runs, iterations, N_regions), dtype=int)

    for n in range(n_runs):
        status = np.zeros(N_regions, dtype=int)
        starting_region = [40, 41, 42, 43]
        status[starting_region] = 1

        status_history[n, 0, :] = status
        for t in range(1, iterations):
            status = SI_simulation_step(
                SC_network, status, best_beta_AD, vulnerability=vulnerability_best
            )
            status_history[n, t, :] = status

    infection_probability = status_history.mean(axis=0)

    corr_t = np.zeros(iterations)
    for t in range(iterations):
        sim_full = infection_probability[t, :]
        if sim_full.std() == 0 or AD_pos_all.std() == 0:
            corr_t[t] = 0.0
        else:
            corr_t[t] = np.corrcoef(sim_full, AD_pos_all)[0, 1]

    R_peaks.append(corr_t.max())

R_peaks = np.array(R_peaks)
print(
    f"average max correlation R = {R_peaks.mean():.3f} ± {R_peaks.std():.3f} "
    f"(number of seeds = {len(seed_list)})"
)

In [19]:
# Observed vs simulated tau load (SUVR).

N_regions = SC_network.shape[0]
vulnerability_best = best_vmin_AD + (best_vmax_AD - best_vmin_AD) * amy_norm 
seed_idx = np.array([40, 41, 42, 43])

# Without the epicenters.
mask = np.ones(N_regions, dtype=bool) 
mask[seed_idx] = True

def simulate_infection_probability(SC, beta, vulnerability, iterations,
                                   n_runs_inner, seed_idx, seed):
    
    np.random.seed(seed) 

    status_history = np.zeros((n_runs_inner, iterations, N_regions), dtype=int)

    for n in range(n_runs_inner):
        status = np.zeros(N_regions, dtype=int)
        status[seed_idx] = 1 

        status_history[n, 0, :] = status
        for t in range(1, iterations):
            status = SI_simulation_step(SC, status, beta, vulnerability=vulnerability)
            status_history[n, t, :] = status

    infection_probability = status_history.mean(axis=0) 
    return infection_probability

# Simulation.
n_outer_runs = 10        # The 10 independent runs.
n_inner_runs = 200      
iterations = 500  

obs_full = AD_pos_all
obs_use = obs_full[mask]     

sim_runs_scaled = []     
y_lines = []             
R_per_run = []

x_line = np.linspace(obs_use.min(), obs_use.max(), 100)

for r in range(n_outer_runs):
    infection_prob = simulate_infection_probability(
        SC_network,
        best_beta_AD,
        vulnerability_best,
        iterations=iterations,
        n_runs_inner=n_inner_runs,
        seed_idx=seed_idx,
        seed=1000 + r,  
    )
    
    sim_vec_full = infection_prob[best_t_AD, :]   
    sim_vec = sim_vec_full[mask]                 

    # Scaling the simulated tau load (SUVR) to the observed SUVR range.
    sim = sim_vec.astype(float)
    obs = obs_use.astype(float)

    sim_mean = sim.mean()
    sim_std = sim.std() + 1e-12
    obs_mean = obs.mean()
    obs_std = obs.std() + 1e-12
    sim_scaled = (sim - sim_mean) / sim_std * obs_std + obs_mean

    sim_runs_scaled.append(sim_scaled)

    slope, intercept = np.polyfit(obs, sim_scaled, 1)
    y_line = slope * x_line + intercept
    y_lines.append(y_line)

    R = np.corrcoef(sim_scaled, obs)[0, 1]
    R_per_run.append(R)

sim_runs_scaled = np.vstack(sim_runs_scaled)   
y_lines = np.vstack(y_lines)                   

sim_mean_scaled = sim_runs_scaled.mean(axis=0)
y_mean = y_lines.mean(axis=0)
y_std = y_lines.std(axis=0)

R_mean = np.mean(R_per_run)
R_std = np.std(R_per_run)

print(f"R_mean={R_mean} and R_std={R_std}")

R_mean=0.1845382156370799 and R_std=0.0068019445402647035
